## Getting started

We build a simple strategy that goes long when overnight return is negative and exits by the end of the day. If price moves against us, we use a stop loss to get out. We have 15 minute bars of Pepsi (PEP) stock prices

In [1]:
# %%checkall
import pandas as pd
import pyqstrat as pq
import numpy as np

# read 1 minute price bars
aapl_file = pq.find_in_subdir('.', 'AAPL.csv.gz')
aapl = pd.read_csv(aapl_file)[['timestamp', 'c']]
aapl.timestamp = pd.to_datetime(aapl.timestamp)
# the date corresponding to each 1minute timestamp
aapl['date'] = aapl.timestamp.values.astype('M8[D]') 
# compute overnight return
aapl['overnight_ret'] = np.where(aapl.date > aapl.date.shift(1), aapl.c / aapl.c.shift(1) - 1, np.nan)
aapl['overnight_ret_negative'] = (aapl.overnight_ret < 0)  # whether overnight return is negative
# mark points just before EOD. We enter a marker order at these points so we have one bar to get filled
aapl['eod'] = np.where(aapl.date.shift(-2) > aapl.date, True, False)   
# if the price drops by 1% after we enter in the morning take our loss and get out
aapl['stop_price'] = np.where(np.isfinite(aapl.overnight_ret), aapl.c * 0.99, np.nan) 
aapl['stop_price'] = aapl.stop_price.fillna(method='ffill')  # fill in the stop price for the rest of the day
aapl['stop'] = np.where(aapl.c < aapl.stop_price, True, False)  # whether we should exit because we are stopped out

strat_builder = pq.StrategyBuilder(data=aapl)   
strat_builder.add_contract('AAPL')
# add the stop price so we can refer to it in
strat_builder.add_series_indicator('stop_price', 'stop_price') 
# convert timestamps from nanoseconds (pandas convention) to minutes so they are easier to view
timestamps = aapl.timestamp.values.astype('M8[m]')  
prices = aapl.c.values
# create a dictionary from contract name=>timestamp => price for use in the price function
price_dict = {'AAPL': {timestamps[i]: prices[i] for i in range(len(timestamps))}}
# create the price function that the strategy will use for looking up prices 
price_function = pq.PriceFuncDict(price_dict=price_dict)
strat_builder.set_price_function(price_function)

# FiniteRiskEntryRule allows us to enter trades and get out with a limited loss when a stop is hit.
# This enters market orders, if you want to use limit orders, set the limit_increment argument
entry_rule = pq.FiniteRiskEntryRule(
    reason_code='POS_OVERNIGHT_RETURN',  # this is useful to know why we entered a trade
    price_func=price_function, 
    long=True,  # whether we enter a long or short position
    percent_of_equity=0.1,  # set the position size so that if the stop is hit, we lose no more than this
    # stop price is used for position sizing.  Also, we will not enter if the price is already below 
    # stop price for long trades and vice versa
    stop_price_ind='stop_price', 
    single_entry_per_day=True)  # if we are stopped out, do we allow re-entry later in the day

# ClosePositionExitRule fully exits a position using either a market or limit order
# In this case, we want to exit at EOD so we are flat overnight
exit_rule_stop = pq.ClosePositionExitRule(   
    reason_code='STOPPED_OUT',
    price_func=price_function)

# Exit when the stop price is reached
exit_rule_eod = pq.ClosePositionExitRule(
    reason_code='EOD',
    price_func=price_function)

# Setup the rules we setup above so they are only called when the columns below in our data dataframe are true
# position filters allow you to choose when the rule runs, "zero" orders it to run only when 
# we don't have a current position, positive and negative similarly run the rule when we
# are currently long or short respectively
strat_builder.add_series_rule('overnight_ret_negative', entry_rule, position_filter='zero')
strat_builder.add_series_rule('eod', exit_rule_eod, position_filter='positive')
strat_builder.add_series_rule('stop', exit_rule_stop, position_filter='positive')

# create the strategy and run it
strategy = strat_builder()
strategy.run()

[2023-10-24 23:04:07.730 __call__] TRADE: 2023-01-13T09:32 AAPL  2023-01-13 09:32:00 qty: 75735 prc: 131.77   order: AAPL 2023-01-13 09:31:00 qty: 75735 POS_OVERNIGHT_RETURN  OrderStatus.OPEN 
[2023-10-24 23:04:07.730 __call__] ORDER: 2023-01-13T15:59 AAPL 2023-01-13 15:59:00 qty: -75735 EOD  OrderStatus.OPEN
[2023-10-24 23:04:07.731 __call__] TRADE: 2023-01-13T16:00 AAPL  2023-01-13 16:00:00 qty: -75735 prc: 134.74   order: AAPL 2023-01-13 15:59:00 qty: -75735 EOD  OrderStatus.OPEN 
[2023-10-24 23:04:07.732 __call__] TRADE: 2023-01-19T09:32 AAPL  2023-01-19 09:32:00 qty: 91096 prc: 134.77   order: AAPL 2023-01-19 09:31:00 qty: 91096 POS_OVERNIGHT_RETURN  OrderStatus.OPEN 
[2023-10-24 23:04:07.733 __call__] ORDER: 2023-01-19T15:59 AAPL 2023-01-19 15:59:00 qty: -91096 EOD  OrderStatus.OPEN
[2023-10-24 23:04:07.733 __call__] TRADE: 2023-01-19T16:00 AAPL  2023-01-19 16:00:00 qty: -91096 prc: 135.59   order: AAPL 2023-01-19 15:59:00 qty: -91096 EOD  OrderStatus.OPEN 
[2023-10-24 23:04:07.7

In [2]:
# Lets evaluate how the strategy did
strategy.df_roundtrip_trades()  

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,AAPL,1.0,2023-01-13 09:32:00,2023-01-13 16:00:00,75735,131.77000,134.73999,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,224932.19265
1,AAPL,1.0,2023-01-19 09:32:00,2023-01-19 16:00:00,91096,134.77000,135.59000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,74698.72000
2,AAPL,1.0,2023-01-20 09:32:00,2023-01-20 16:00:00,96311,134.57501,137.87001,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,317344.74500
3,AAPL,1.0,2023-01-24 09:32:00,2023-01-24 16:00:00,114948,141.15102,142.66000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,173454.23304
4,AAPL,1.0,2023-01-25 09:32:00,2023-01-25 16:00:00,128153,139.56000,141.79000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,285781.19000
5,AAPL,1.0,2023-01-27 09:32:00,2023-01-27 16:00:00,144281,144.23999,145.93000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,243836.33281
6,AAPL,1.0,2023-01-30 09:32:00,2023-01-30 10:37:00,159650,145.34500,143.65001,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-270605.15350


In [4]:
# Lets evaluate how the strategy did
metrics = strategy.evaluate_returns(plot=pq.has_display())

,gmean,amean,std,shrp,srt,k,calmar,mar,mdd_pct,mdd_dates,dd_3y_pct,dd_3y_timestamps,up_dwn,2023
,1.359e+04,10.05,0.08558,7.399,24.91,5.963,86.18,86.18,0.1166,2023-01-27/2023-01-30,0.1166,2023-01-27/2023-01-30,6/1/0.857,1.359e+04


2023-01-27T00:00 2023-01-30T00:00
